# Создание системы RAG с помощью Gemma, MongoDB и моделей с открытым исходным кодом

Автор:  [Richmond Alake](https://huggingface.co/RichmondMongo)

## Шаг 1: Установка библиотек


Приведенная ниже последовательность команд оболочки устанавливает библиотеки для использования открытых больших языковых моделей (LLM), моделей эмбеддингов и функций взаимодействия с базами данных. Эти библиотеки упрощают разработку системы RAG, снижая сложность до небольшого количества кода:


- PyMongo: Библиотека Python для взаимодействия с MongoDB, позволяющая подключаться к кластеру и запрашивать данные, хранящиеся в коллекциях и документах.
- Pandas: Предоставляет структуру данных для эффективной обработки и анализа данных с помощью Python.
- Hugging Face datasets: Содержит наборы аудио-, визуальных и текстовых данных.
- Hugging Face Accelerate: Абстрагируется от сложности написания кода, использующего аппаратные ускорители, таких как GPU. Accelerate используется в этой реализации для исполнения модели Gemma на ресурсах GPU.
- Hugging Face Transformers: Доступ к обширной коллекции предварительно обученных моделей
- Hugging Face Sentence Transformers: Предоставляет доступ к эмбеддингам предложений, текстов и изображений.

In [ ]:
!pip install datasets pandas pymongo sentence_transformers
!pip install -U transformers
# Установите библиотеку ниже, если вы используете GPU
!pip install accelerate

## Шаг 2: Сбор и подготовка данных


Данные, используемые в этом руководстве, взяты из наборов данных Hugging Face, а именно 
[AIatMongoDB/embedded_movies dataset](https://huggingface.co/datasets/AIatMongoDB/embedded_movies). 

In [2]:
# Загрузка набора данных
from datasets import load_dataset
import pandas as pd

# https://huggingface.co/datasets/AIatMongoDB/embedded_movies
dataset = load_dataset("AIatMongoDB/embedded_movies")

# Конвертация набора данных в фрейм данных Pandas
dataset_df = pd.DataFrame(dataset["train"])

dataset_df.head(5)

,num_mflix_comments,genres,countries,directors,fullplot,writers,awards,runtime,type,rated,metacritic,poster,languages,imdb,plot,cast,plot_embedding,title
0,0,[Action],[USA],"[Louis J. Gasnier, Donald MacKenzie]",Young Pauline is left a lot of money when her ...,"[Charles W. Goddard (screenplay), Basil Dickey...","{'nominations': 0, 'text': '1 win.', 'wins': 1}",199.0,movie,None,NaN,https://m.media-amazon.com/images/M/MV5BMzgxOD...,[English],"{'id': 4465, 'rating': 7.6, 'votes': 744}",Young Pauline is left a lot of money when her ...,"[Pearl White, Crane Wilbur, Paul Panzer, Edwar...","[0.00072939653, -0.026834568, 0.013515796, -0....",The Perils of Pauline
1,0,"[Comedy, Short, Action]",[USA],"[Alfred J. Goulding, Hal Roach]",As a penniless man worries about how he will m...,[H.M. Walker (titles)],"{'nominations': 1, 'text': '1 nomination.', 'w...",22.0,movie,TV-G,NaN,https://m.media-amazon.com/images/M/MV5BNzE1OW...,[English],"{'id': 10146, 'rating': 7.0, 'votes': 639}",A penniless young man tries to save an heiress...,"[Harold Lloyd, Mildred Davis, 'Snub' Pollard, ...","[-0.022837115, -0.022941574, 0.014937485, -0.0...",From Hand to Mouth
2,0,"[Action, Adventure, Drama]",[USA],[Herbert Brenon],"Michael ""Beau"" Geste leaves England in disgrac...","[Herbert Brenon (adaptation), John Russell (ad...","{'nominations': 0, 'text': '1 win.', 'wins': 1}",101.0,movie,None,NaN,None,[English],"{'id': 16634, 'rating': 6.9, 'votes': 222}","Michael ""Beau"" Geste leaves England in disgrac...","[Ronald Colman, Neil Hamilton, Ralph Forbes, A...","[0.00023330493, -0.028511643, 0.014653289, -0....",Beau Geste
3,1,"[Adventure, Action]",[USA],[Albert Parker],A nobleman vows to avenge the death of his fat...,"[Douglas Fairbanks (story), Jack Cunningham (a...","{'nominations': 0, 'text': '1 win.', 'wins': 1}",88.0,movie,None,NaN,https://m.media-amazon.com/images/M/MV5BMzU0ND...,None,"{'id': 16654, 'rating': 7.2, 'votes': 1146}","Seeking revenge, an athletic young man joins t...","[Billie Dove, Tempe Pigott, Donald Crisp, Sam ...","[-0.005927917, -0.033394486, 0.0015323418, -0....",The Black Pirate
4,0,"[Action, Comedy, Romance]",[USA],[Sam Taylor],"The Uptown Boy, J. Harold Manners (Lloyd) is a...","[Ted Wilde (story), John Grey (story), Clyde B...","{'nominations': 1, 'text': '1 nomination.', 'w...",58.0,movie,PASSED,NaN,https://m.media-amazon.com/images/M/MV5BMTcxMT...,[English],"{'id': 16895, 'rating': 7.6, 'votes': 918}",An irresponsible young millionaire changes his...,"[Harold Lloyd, Jobyna Ralston, Noah Young, Jim...","[-0.0059373598, -0.026604708, -0.0070914757, -...",For Heaven's Sake


Операции в следующем фрагменте кода направлены на обеспечение целостности и качества данных. 
1. Первый процесс гарантирует, что атрибут `fullplot` каждой точки данных не пуст, поскольку это первичные данные, которые мы используем в процессе эмбеддингов. 
2. Этот шаг также гарантирует, что мы удалим атрибут `plot_embedding` из всех точек данных, поскольку он будет заменен новыми эмбеддингами, созданными с помощью другой модели эмбеддингов, `gte-large`.

In [3]:
# Подготовка данных

# Удалим точки данных, в которой отсутствует столбец графика
dataset_df = dataset_df.dropna(subset=["fullplot"])
print("\nNumber of missing values in each column after removal:")
print(dataset_df.isnull().sum())

# Удаляем plot_embedding из каждой точки данных в наборе данных, так как мы собираемся создать новые эмбеддинги с помощью модели эмбеддингов c открытым исходным кодом из Hugging Face
dataset_df = dataset_df.drop(columns=["plot_embedding"])
dataset_df.head(5)


Number of missing values in each column after removal:
num_mflix_comments      0
genres                  0
countries               0
directors              12
fullplot                0
writers                13
awards                  0
runtime                14
type                    0
rated                 279
metacritic            893
poster                 78
languages               1
imdb                    0
plot                    0
cast                    1
plot_embedding          1
title                   0
dtype: int64


,num_mflix_comments,genres,countries,directors,fullplot,writers,awards,runtime,type,rated,metacritic,poster,languages,imdb,plot,cast,title
0,0,[Action],[USA],"[Louis J. Gasnier, Donald MacKenzie]",Young Pauline is left a lot of money when her ...,"[Charles W. Goddard (screenplay), Basil Dickey...","{'nominations': 0, 'text': '1 win.', 'wins': 1}",199.0,movie,None,NaN,https://m.media-amazon.com/images/M/MV5BMzgxOD...,[English],"{'id': 4465, 'rating': 7.6, 'votes': 744}",Young Pauline is left a lot of money when her ...,"[Pearl White, Crane Wilbur, Paul Panzer, Edwar...",The Perils of Pauline
1,0,"[Comedy, Short, Action]",[USA],"[Alfred J. Goulding, Hal Roach]",As a penniless man worries about how he will m...,[H.M. Walker (titles)],"{'nominations': 1, 'text': '1 nomination.', 'w...",22.0,movie,TV-G,NaN,https://m.media-amazon.com/images/M/MV5BNzE1OW...,[English],"{'id': 10146, 'rating': 7.0, 'votes': 639}",A penniless young man tries to save an heiress...,"[Harold Lloyd, Mildred Davis, 'Snub' Pollard, ...",From Hand to Mouth
2,0,"[Action, Adventure, Drama]",[USA],[Herbert Brenon],"Michael ""Beau"" Geste leaves England in disgrac...","[Herbert Brenon (adaptation), John Russell (ad...","{'nominations': 0, 'text': '1 win.', 'wins': 1}",101.0,movie,None,NaN,None,[English],"{'id': 16634, 'rating': 6.9, 'votes': 222}","Michael ""Beau"" Geste leaves England in disgrac...","[Ronald Colman, Neil Hamilton, Ralph Forbes, A...",Beau Geste
3,1,"[Adventure, Action]",[USA],[Albert Parker],A nobleman vows to avenge the death of his fat...,"[Douglas Fairbanks (story), Jack Cunningham (a...","{'nominations': 0, 'text': '1 win.', 'wins': 1}",88.0,movie,None,NaN,https://m.media-amazon.com/images/M/MV5BMzU0ND...,None,"{'id': 16654, 'rating': 7.2, 'votes': 1146}","Seeking revenge, an athletic young man joins t...","[Billie Dove, Tempe Pigott, Donald Crisp, Sam ...",The Black Pirate
4,0,"[Action, Comedy, Romance]",[USA],[Sam Taylor],"The Uptown Boy, J. Harold Manners (Lloyd) is a...","[Ted Wilde (story), John Grey (story), Clyde B...","{'nominations': 1, 'text': '1 nomination.', 'w...",58.0,movie,PASSED,NaN,https://m.media-amazon.com/images/M/MV5BMTcxMT...,[English],"{'id': 16895, 'rating': 7.6, 'votes': 918}",An irresponsible young millionaire changes his...,"[Harold Lloyd, Jobyna Ralston, Noah Young, Jim...",For Heaven's Sake


## Шаг 3: Генерация эмбеддингов

**В ячейке кода описаны следующие шаги:**.
1. Импортируйте класс `SentenceTransformer` для доступа к моделям эмбеддингов.
2. Загрузим модель эмбеддингов с помощью конструктора `SentenceTransformer` для инстанцирования модели эмбеддингов `gte-large`.
3. Определим функцию `get_embedding`, которая принимает на вход текстовую строку и возвращает список значений с плавающей точкой, представляющих эмбеддинги. Сначала функция проверяет, не пуст ли входной текст (после удаления пробельных символов). Если текст пуст, она возвращает пустой список. В противном случае она генерирует эмбеддинги, используя загруженную модель.
4. Генерируем эмбеддинги, применяя функцию `get_embedding` к столбцу "fullplot" фрейма данных `dataset_df`, генерируя эмбеддинги для каждого сюжета фильма. Полученный список эмбеддингов присваивается новому столбцу с именем embedding.

*Примечание: нет необходимости разбивать текст на фрагменты (chunk) в полном описании сюжета, так как мы можем гарантировать, что длина текста остается в пределах допустимого диапазона*.



In [4]:
from sentence_transformers import SentenceTransformer

# https://huggingface.co/thenlper/gte-large
embedding_model = SentenceTransformer("thenlper/gte-large")


def get_embedding(text: str) -> list[float]:
    if not text.strip():
        print("Attempted to get embedding for empty text.")
        return []

    embedding = embedding_model.encode(text)

    return embedding.tolist()


dataset_df["embedding"] = dataset_df["fullplot"].apply(get_embedding)

dataset_df.head()

,num_mflix_comments,genres,countries,directors,fullplot,writers,awards,runtime,type,rated,metacritic,poster,languages,imdb,plot,cast,title,embedding
0,0,[Action],[USA],"[Louis J. Gasnier, Donald MacKenzie]",Young Pauline is left a lot of money when her ...,"[Charles W. Goddard (screenplay), Basil Dickey...","{'nominations': 0, 'text': '1 win.', 'wins': 1}",199.0,movie,None,NaN,https://m.media-amazon.com/images/M/MV5BMzgxOD...,[English],"{'id': 4465, 'rating': 7.6, 'votes': 744}",Young Pauline is left a lot of money when her ...,"[Pearl White, Crane Wilbur, Paul Panzer, Edwar...",The Perils of Pauline,"[-0.009285838343203068, -0.005062104668468237,..."
1,0,"[Comedy, Short, Action]",[USA],"[Alfred J. Goulding, Hal Roach]",As a penniless man worries about how he will m...,[H.M. Walker (titles)],"{'nominations': 1, 'text': '1 nomination.', 'w...",22.0,movie,TV-G,NaN,https://m.media-amazon.com/images/M/MV5BNzE1OW...,[English],"{'id': 10146, 'rating': 7.0, 'votes': 639}",A penniless young man tries to save an heiress...,"[Harold Lloyd, Mildred Davis, 'Snub' Pollard, ...",From Hand to Mouth,"[-0.0024393785279244184, 0.02309592440724373, ..."
2,0,"[Action, Adventure, Drama]",[USA],[Herbert Brenon],"Michael ""Beau"" Geste leaves England in disgrac...","[Herbert Brenon (adaptation), John Russell (ad...","{'nominations': 0, 'text': '1 win.', 'wins': 1}",101.0,movie,None,NaN,None,[English],"{'id': 16634, 'rating': 6.9, 'votes': 222}","Michael ""Beau"" Geste leaves England in disgrac...","[Ronald Colman, Neil Hamilton, Ralph Forbes, A...",Beau Geste,"[0.012204292230308056, -0.01145575474947691, -..."
3,1,"[Adventure, Action]",[USA],[Albert Parker],A nobleman vows to avenge the death of his fat...,"[Douglas Fairbanks (story), Jack Cunningham (a...","{'nominations': 0, 'text': '1 win.', 'wins': 1}",88.0,movie,None,NaN,https://m.media-amazon.com/images/M/MV5BMzU0ND...,None,"{'id': 16654, 'rating': 7.2, 'votes': 1146}","Seeking revenge, an athletic young man joins t...","[Billie Dove, Tempe Pigott, Donald Crisp, Sam ...",The Black Pirate,"[0.004541348200291395, -0.0006100579630583525,..."
4,0,"[Action, Comedy, Romance]",[USA],[Sam Taylor],"The Uptown Boy, J. Harold Manners (Lloyd) is a...","[Ted Wilde (story), John Grey (story), Clyde B...","{'nominations': 1, 'text': '1 nomination.', 'w...",58.0,movie,PASSED,NaN,https://m.media-amazon.com/images/M/MV5BMTcxMT...,[English],"{'id': 16895, 'rating': 7.6, 'votes': 918}",An irresponsible young millionaire changes his...,"[Harold Lloyd, Jobyna Ralston, Noah Young, Jim...",For Heaven's Sake,"[-0.0022256041411310434, 0.011567804962396622,..."


## Шаг 4: Настройка и подключение к базе данных

MongoDB выступает в качестве как операционной, так и векторной базы данных. Она предлагает решение базы данных, которое эффективно хранит, запрашивает и извлекает эмбеддинги векторов - преимущества этого решения заключаются в простоте обслуживания, управления и стоимости базы данных.

**Чтобы создать новую базу данных MongoDB, настройте кластер баз данных:**

1. Перейдите на официальный сайт MongoDB и зарегистрируйте [бесплатную учетную запись MongoDB Atlas](https://www.mongodb.com/cloud/atlas/register?utm_campaign=devrel&utm_source=community&utm_medium=cta&utm_content=Partner%20Cookbook&utm_term=richmond.alake), или для существующих пользователей - [войдите в MongoDB Atlas](https://account.mongodb.com/account/login?utm_campaign=devrel&utm_source=community&utm_medium=cta&utm_content=Partner%20Cookbook&utm_term=richmond.alakee).

2. Выберите параметр 'Database' на левой панели, что приведет к переходу на страницу развертывания базы данных, где есть спецификация развертывания любого существующего кластера. Создайте новый кластер базы данных, нажав на кнопку "+Create".

3.   Выберите все применимые конфигурации для кластера баз данных. Когда все параметры конфигурации выбраны, нажмите кнопку “Create Cluster”, чтобы развернуть только что созданный кластер. MongoDB также позволяет создавать бесплатные кластеры на вкладке "Shared Tab".

 *Примечание: Не забудьте внести в белый список IP-адрес хоста Python или 0.0.0.0/0 для любого IP-адреса при создании пробного варианта*.

4. После успешного создания и развертывания кластера он становится доступным на странице ‘Database Deployment’.

5. Нажмите на кнопку “Connect” кластера, чтобы увидеть возможность установки соединения с кластером через различные языковые драйверы.

6. В этом руководстве требуется только URI (уникальный идентификатор ресурса) кластера. Скопируйте этот URI и вставьте его в окружение Google Colabs Secrets в переменную с именем `MONGO_URI` или добавьте его в файл .env или аналогичный.


### 4.1 Настройка базы данных и коллекции

Прежде чем двигаться дальше, убедитесь, что выполнены следующие предварительные условия
- Кластер баз данных задан как MongoDB Atlas
- Получен URI вашего кластера

За помощью в настройке кластера баз данных и получении URI обращайтесь к нашему руководству по [настройке кластера MongoDB](https://www.mongodb.com/docs/guides/atlas/cluster/) и [получение строки подключения](https://www.mongodb.com/docs/guides/atlas/connection-string/)

После создания кластера создайте базу данных и коллекцию в кластере MongoDB Atlas, нажав + Create Database на странице описания кластера.  

Вот руководство по [созданию базы данных и коллекции](https://www.mongodb.com/basics/create-database)

**База данных будет называться `movies`.**

**Коллекция будет называться `movie_collection_2`.**




## Шаг 5: Создание индекса векторного поиска

На этом этапе убедитесь, что ваш векторный индекс создан через MongoDB Atlas.

Этот шаг является обязательным для проведения эффективного и точного векторного поиска на основе векторов эмбеддингов, хранящихся в документах коллекции `movie_collection_2`. 

Создание индекса векторного поиска позволяет эффективно просматривать документы и извлекать документы с эмбеддингами, соответствующими эмбеддингам запроса, на основе векторного сходства (vector similarity). 

Перейдите сюда, чтобы узнать больше о [Индексе векторного поиска MongoDB](https://www.mongodb.com/docs/atlas/atlas-search/field-types/knn-vector/).


```
{
 "fields": [{
     "numDimensions": 1024,
     "path": "embedding",
     "similarity": "cosine",
     "type": "vector"
   }]
}

```

Значение `1024` в поле numDimension соответствует размеру вектора, сгенерированного моделью эмбеддингов gte-large. Если вы используете модели эмбеддингов `gte-base` или `gte-small`, значение numDimension в индексе векторного поиска должно быть задано равным 768 и 384 соответственно.


## Шаг 6: Установите соединение с данными

Приведенный ниже фрагмент кода также использует PyMongo для создания объекта-клиента MongoDB, представляющего соединение с кластером и обеспечивающего доступ к его базам данных и коллекциям.


In [ ]:
import pymongo
from google.colab import userdata


def get_mongo_client(mongo_uri):
    """Установите соединение с MongoDB."""
    try:
        client = pymongo.MongoClient(mongo_uri)
        print("Connection to MongoDB successful")
        return client
    except pymongo.errors.ConnectionFailure as e:
        print(f"Connection failed: {e}")
        return None


mongo_uri = userdata.get("MONGO_URI")
if not mongo_uri:
    print("MONGO_URI not set in environment variables")

mongo_client = get_mongo_client(mongo_uri)

# Загрузка данных в MongoDB
db = mongo_client["movies"]
collection = db["movie_collection_2"]

Connection to MongoDB successful


In [6]:
# Удаление всех существующих записей в коллекции
collection.delete_many({})

DeleteResult({'n': 1452, 'electionId': ObjectId('7fffffff000000000000000c'), 'opTime': {'ts': Timestamp(1708554945, 1452), 't': 12}, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1708554945, 1452), 'signature': {'hash': b'\x99\x89\xc0\x00Cn!\xd6\xaf\xb3\x96\xdf\xc3\xda\x88\x11\xf5\t\xbd\xc0', 'keyId': 7320226449804230661}}, 'operationTime': Timestamp(1708554945, 1452)}, acknowledged=True)

Загрузка данных в коллекцию MongoDB из pandas DataFrame - это простой процесс, который можно эффективно выполнить, преобразовав DataFrame в словари, а затем используя метод `insert_many` в коллекции для передачи преобразованных записей набора данных.


In [7]:
documents = dataset_df.to_dict("records")
collection.insert_many(documents)

print("Data ingestion into MongoDB completed")

Data ingestion into MongoDB completed


## Шаг 7: Выполнение векторного поиска по запросам пользователей

На следующем этапе реализуется функция, которая возвращает результат векторного поиска, генерируя эмбеддингов запроса и определяя конвейер агрегации MongoDB. 

Конвейер, состоящий из этапов `$vectorSearch` и `$project`, выполняет запросы, используя сгенерированный вектор, и форматирует результаты так, чтобы они включали только необходимую информацию, такую как сюжет, название и жанры, а также включали оценку поиска для каждого результата.

In [8]:
def vector_search(user_query, collection):
    """
    Perform a vector search in the MongoDB collection based on the user query.

    Args:
    user_query (str): The user's query string.
    collection (MongoCollection): The MongoDB collection to search.

    Returns:
    list: A list of matching documents.
    """

    # Генерация эмбеддингов для пользовательского запроса
    query_embedding = get_embedding(user_query)

    if query_embedding is None:
        return "Invalid query or embedding generation failed."

    # Определение конвейера векторного поиска
    pipeline = [
        {
            "$vectorSearch": {
                "index": "vector_index",
                "queryVector": query_embedding,
                "path": "embedding",
                "numCandidates": 150,  # Количество кандидатов для рассмотрения
                "limit": 4,  # Возврат 4 лучших совпадений
            }
        },
        {
            "$project": {
                "_id": 0,  # Исключите поле _id
                "fullplot": 1,  # Включить поле plot
                "title": 1,  # Включить поле  title
                "genres": 1,  # Включить поле  genres
                "score": {"$meta": "vectorSearchScore"},  # Включите оценку поиска
            }
        },
    ]

    # Execute the search
    results = collection.aggregate(pipeline)
    return list(results)

## Шаг 8: Обработка пользовательских запросов и загрузка Gemma


In [ ]:
def get_search_result(query, collection):

    get_knowledge = vector_search(query, collection)

    search_result = ""
    for result in get_knowledge:
        search_result += f"Title: {result.get('title', 'N/A')}, Plot: {result.get('fullplot', 'N/A')}\n"

    return search_result

In [13]:
# Выполнение запроса с извлечением источников
query = "What is the best romantic movie to watch and why?"
source_information = get_search_result(query, collection)
combined_information = f"Query: {query}\nContinue to answer the query by using the Search Results:\n{source_information}."

print(combined_information)

Query: What is the best romantic movie to watch and why?
Continue to answer the query by using the Search Results:
Title: Shut Up and Kiss Me!, Plot: Ryan and Pete are 27-year old best friends in Miami, born on the same day and each searching for the perfect woman. Ryan is a rookie stockbroker living with his psychic Mom. Pete is a slick surfer dude yet to find commitment. Each meets the women of their dreams on the same day. Ryan knocks heads in an elevator with the gorgeous Jessica, passing out before getting her number. Pete falls for the insatiable Tiara, but Tiara's uncle is mob boss Vincent Bublione, charged with her protection. This high-energy romantic comedy asks to what extent will you go for true love?
Title: Pearl Harbor, Plot: Pearl Harbor is a classic tale of romance set during a war that complicates everything. It all starts when childhood friends Rafe and Danny become Army Air Corps pilots and meet Evelyn, a Navy nurse. Rafe falls head over heels and next thing you know

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")
# Для выполнения на CPU раскоментируйте код ниже 👇🏽
# model = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it")
# Для выполнения на GPU раскоментируйте код ниже 👇🏽
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it", device_map="auto")

In [16]:
# Передача тензоров на GPU
input_ids = tokenizer(combined_information, return_tensors="pt").to("cuda")
response = model.generate(**input_ids, max_new_tokens=500)
print(tokenizer.decode(response[0]))

<bos>Query: What is the best romantic movie to watch and why?
Continue to answer the query by using the Search Results:
Title: Shut Up and Kiss Me!, Plot: Ryan and Pete are 27-year old best friends in Miami, born on the same day and each searching for the perfect woman. Ryan is a rookie stockbroker living with his psychic Mom. Pete is a slick surfer dude yet to find commitment. Each meets the women of their dreams on the same day. Ryan knocks heads in an elevator with the gorgeous Jessica, passing out before getting her number. Pete falls for the insatiable Tiara, but Tiara's uncle is mob boss Vincent Bublione, charged with her protection. This high-energy romantic comedy asks to what extent will you go for true love?
Title: Pearl Harbor, Plot: Pearl Harbor is a classic tale of romance set during a war that complicates everything. It all starts when childhood friends Rafe and Danny become Army Air Corps pilots and meet Evelyn, a Navy nurse. Rafe falls head over heels and next thing you